<div style="font-size: 20px">
Two-phase flash calculation

<div style="font-size: 16px">
1. specify the properties of the components, including critical T, critical P, acentric factor and molecular mass

In [10]:
import numpy as np
# load the properties of each components with corresponding index
def f(component, property):
    properties = [["CO2",   "N2",    "H2S",   "C1",   "C2",    "C3",  "nC4", "iC4",  "nC5",   "nC6",   "nC7"],
                  [304.10,  126.20,  373.53,  190.58, 305.32,  369.8, 425.2, 408.2,  469.7,   507.5,   540.3],  # T_c [K]
                  [73.75,   34.00,   89.63,   46.04,  48.72,   42.5,  38,    36.5,   33.7,    30.1,    27.4],   # p_c [bar]
                  [0.239,   0.0377,  0.0942,  0.012,  0.0995,  0.153, 0.199, 0.183,  0.251,   0.299,   0.349],  # acentric factor [-]
                  [44.01,   28.013,  34.076,  16.043, 30.07,   "",    "",    "",     "",      "",      ""]]     # molecular mass [g/mol]

    prop = ["Tc", "Pc", "ac", "Mw"]
    index1 = prop.index(property) + 1
    index2 = properties[0][:].index(component)
    c = properties[index1][index2]

    return c

<div style="font-size: 16px">
2. define the function of calculating partitioning coefficients based on Wilson's equation (true for low pressure)

<img style="float: center;" src="Practical3/partitioning coefficient.jpg" width="25%">

<div style="font-size: 16px">
Note here we need to prepare the reduced P and T

In [11]:
def vapour_liquid(p, T, z, components): # components -> component name list
    NC = np.size(z)
    K_VL = np.zeros(NC)
    for i in range(0, NC):
        K_VL[i] = f(components[i], "Pc") / p * np.exp(5.373 * (1 + f(components[i], "ac")) *
                                                           (1 - f(components[i], "Tc") / T))  # K_i = x_V/x_L

    return K_VL

<div style="font-size: 16px">
3. define a function to solve Rachford-Rice equation to find V

<img style="float: center;" src="Practical3/Rachford-Rice equation.jpg" width="25%">

In [12]:
# two phase negative flash
def two_phase_negative_flash(z, K):
    # Two-phase negative flash procedure
    maxk = np.amax(K)
    mink = np.amin(K)
    eps = 1E-10
    V_low = 1 / (1 - maxk) + eps
    V_high = 1 / (1 - mink) - eps
    V_mid = (V_low + V_high) / 2

    def obj(z, K, V):
#         f = (z*(1-K))/(1+(K-1)*V)
        f = (z*(K-1))/(1+(K-1)*V)
        f = np.sum(f)
        return f

    iter = 0
    while np.absolute(obj(z, K, V_mid)) > 1E-8:
        if obj(z, K, V_mid) * obj(z, K, V_low) < 0:
            V_high = V_mid
        else:
            V_low = V_mid
        V_mid = (V_high + V_low)/2
        iter = iter+1
    v = np.array([1-V_mid, V_mid])  # V_mid is molar fraction of non-reference phase
    x1 = z/(v[1]*(K-1)+1)
    x2 = K*x1
    x = np.block([[x1], [x2]])
#     print('iter negtive flash %s' % iter)
    return v, x

<div style="font-size: 16px">
4. define a function for fugacity calculation

   step 1: calculate $\alpha$
   
   <img style="float: center;" src="Practical3/calculate alpha.JPG" width="35%">
   
   step 2: use combining rule and mixing rule to calculate a and b, and prepare the coefficients A and B (see step 3) in cubic form of EoS 
   
   <img style="float: center;" src="Practical3/combining and mixing.JPG" width="35%">
   
   step 3: find the roots of cubic EoS (calculate Z)
   
   <img style="float: center;" src="Practical3/cubic EoS.JPG" width="35%">
   
   step 4: conducte fugacity calculation
   
   <img style="float: center;" src="Practical3/fugacity.jpg" width="35%">
   
   

In [18]:
# update K values
def k_update_vapour_liquid(p, T, x, components, phase):
    Peneloux = True  # perform Peneloux correction for molar volume

    # Peng-Robinson EoS
    NC = np.size(x)
    R = 8.3145E-5

    # attraction parameter
    ai = np.zeros(NC)
    for i in range(0, np.size(ai)):
        C_a = 1
        Tc = f(components[i], "Tc")
        Pc = f(components[i], "Pc")
        ac = f(components[i], "ac")
        kappa = 0.37464 + 1.54226*ac - 0.26992*ac**2
        alpha = (1+C_a*kappa*(1-np.sqrt(T/Tc)))**2
        ai[i] = 0.45724*R**2*Tc**2/Pc*alpha
    dij = [["CO2",   "N2",    "H2S",   "C1",  "C2",    "C3",  "iC4",  "nC4",  "nC5",  "nC6",  "nC7"],  # from Aspen plus (DOI 10.1016/j.fluid.2016.06.012)
           [0,       0,       0.1200,  0.105,  0.130,  0.125, 0.1154, 0.1154, 0.1154, 0.1154, 0.115],
           [0,       0,       0,       0.025,  0.010,  0.090, 0.1040, 0.1040, 0.1040, 0.1040, 0.110],
           [0.1200,  0,       0,       0.080,  0.070,  0.070, 0.0544, 0.0544, 0.0544, 0.0544, 0.050],
           [0.105,   0.025,   0.080,   0,      0,      0,     0,      0,      0,      0,      0],
           [0.130,   0.010,   0.070,   0,      0,      0,     0,      0,      0,      0,      0],
           [0.125,   0.090,   0.070,   0,      0,      0,     0,      0,      0,      0,      0],
           [0.1154,  0.104,   0.0544,  0,      0,      0,     0,      0,      0,      0,      0],
           [0.1154,  0.104,   0.0544,  0,      0,      0,     0,      0,      0,      0,      0],
           [0.1154,  0.104,   0.0544,  0,      0,      0,     0,      0,      0,      0,      0],
           [0.1154,  0.104,   0.0544,  0,      0,      0,     0,      0,      0,      0,      0],
           [0.115,   0.110,   0.050,   0,      0,      0,     0,      0,      0,      0,      0]]  # binary interaction parameters. dij == dji

    aij = np.zeros((NC, NC))
    a = 0
    for i in range(0, NC):
        indexi = dij[0][:].index(components[i]) + 1  #
        for j in range(0, NC):
            indexj = dij[0][:].index(components[j])
            aij[i, j] = (ai[i])**(1/2)*(ai[j])**(1/2)*(1-dij[indexi][indexj])
            a += aij[i, j]*x[i]*x[j]
    A = a*p/(R**2*T**2)

    # repulsion parameter
    bi = np.zeros(NC)
    b = 0
    for i in range(0, NC):
        C_b = 1
        Tc = f(components[i], "Tc")
        Pc = f(components[i], "Pc")
        bi[i] = C_b*0.0778*R*Tc/Pc
        b += bi[i]*x[i]

    if Peneloux:
        # Peneloux volume shift parameter (1982)
        c = 0
        for i in range(0, NC):
            z_ra = 0.29056 - 0.08775 * f(components[i], "ac")
            c += x[i]*(0.50033 * R * f(components[i], "Tc") / f(components[i], "Pc") * (0.25969 - z_ra))

        b = b - c  # Peneloux correction for b and V
    B = b*p/(R*T)

    # solve for compressibility Z
    Z = np.roots([1, -(1-B), A-3*B**2-2*B, -(A*B-B**2-B**3)])  # 3 real roots: 2-phase region; 1 real root: supercritical??

    # interpret solution for Z and calculate fugacity coefficients
    if np.sum(np.imag(Z) == 0) == 1:  # supercritical??
        index = np.nonzero(np.imag(Z) == 0)  # find real root
        Z = np.real(Z[index])[0]   # Z reduces to only the real root
    else:
        if phase == "V":
            Z = np.amax(Z)
        elif phase == "L":
            Z = np.amin(Z)
        else:
            Z = np.amax(Z)
    # print(phase, Z)

    # fugacity coefficients for either V or L phase
    phi_c = np.zeros(NC)
    for k in range(0, NC):  # for 2-phase: fugacity coefficient for each phase, based on x and y input
        phi_c[k] = np.exp(bi[k] / b * (Z - 1) - np.log(Z - B) - A / (2.828 * B) * (2 * np.sum(x*aij[:, k])
                                                    / a - bi[k] / b) * np.log((Z + 2.414 * B) / (Z - 0.414 * B)))


    Vm = Z*R*T/p  # molar volume
    if Peneloux:
        Vm = Vm - c   # Peneloux correction

    return phi_c

<div style="font-size: 16px">
5. conduct two-phase flash calculation using the functions defined above

In [19]:
p = 50
T = 300
phases = ["V", "L"]
components = ["C1", "iC4", "nC5"]
z = [0.4, 0.3]
z = np.append(z, 1-np.sum(z))

NC = np.size(components)
NP = np.size(phases)

# Initial K-values
K = np.zeros((NP-1, NC))
K[0, :] = vapour_liquid(p, T, z, components)
K = 1/K  # K_VL from vapour_liquid file = x_iV/x_iL -> base phase should be in denominator, so invert


converged = 0
iter = 0

while converged == 0:
    # Flash loop
    V, x = two_phase_negative_flash(z, K)
    if np.sum(V < 0) == 1:  # negative phase fractions
        x = np.zeros((NP, NC))
        for i in range(0, np.size(V)):
            if V[i] < 0:
                V[i] = 0
            else:
                V[i] = 1
                x[i, :] = z[:]  # single phase -> mole fractions of present phase equal to composition

    if np.count_nonzero(V) == 1:  # single phase
        converged = 1
    else:
        # Update K-values
        # Base phase fugacity
        phi_c0 = k_update_vapour_liquid(p, T, x[0, :], components, phases[0])

        # Phase i fugacity & K-values
        phi_c = k_update_vapour_liquid(p, T, x[1, :], components, phases[1])
        K1 = phi_c0 / phi_c
        # print(K1)

        if np.amax(np.abs(K1-K)) < 1E-10:  # converged to equal fugacity
            converged = 1

        K = K1

    iter += 1

print("V, x:", V, x)
print('K:', K)
print("iterations:", iter)

V, x: [0.12990488 0.87009512] [[0.91719682 0.06169123 0.02111195]
 [0.3227827  0.33557941 0.34163788]]
K: [ 0.35192305  5.43966141 16.18220048]
iterations: 21
